In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from models.tokenization import *
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from models.model_architecture import *
from models.train_eval import *
from tqdm import tqdm, tqdm_notebook
parent_path='../Data/New_Data_15-06-2020/'

In [6]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')


In [4]:
torch.cuda.set_device(1)

In [8]:
params={'model_path':'xlm-roberta-base',
        'max_length':128,
        'batch_size':6,
        'weights':[1.0,1.0],
        'data_path':parent_path+'Fearspeech_data_final.pkl',
        'max_sentences_per_doc':10,
        'transformer_type':'lstm_transformer',
        'device':'cuda',
        'learning_rate':2e-5,
        'epsilon':1e-8,
        'random_seed':2,
        'epochs':5
       }

In [6]:
train_phase(params)

100%|██████████| 4782/4782 [00:36<00:00, 216.16it/s]


Loading tokenizer...


OSError: Can't load config for 'xlm-roberta-base'. Make sure that:

- 'xlm-roberta-base' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'xlm-roberta-base' is the correct path to a directory containing a config.json file



In [19]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
    

I0924 09:30:24.217592 140120072415040 connectionpool.py:700] Starting new HTTPS connection (1): s3.amazonaws.com
I0924 09:30:25.342872 140120072415040 connectionpool.py:700] Starting new HTTPS connection (1): s3.amazonaws.com


In [ ]:
temp=tokenizer.tokenize("I am goof")



In [ ]:
temp=['<s>']+temp+['</s>']

In [ ]:
tokenizer.convert_tokens_to_ids(temp)

In [3]:
from transformers import AutoConfig
config=AutoConfig.from_pretrained("xlm-roberta-base")

OSError: Can't load config for 'xlm-roberta-base'. Make sure that:

- 'xlm-roberta-base' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'xlm-roberta-base' is the correct path to a directory containing a config.json file



In [ ]:
config

In [9]:
annotated_df=pd.read_pickle(params['data_path'])
params_preprocess={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': False}
list_sents=[preprocess_sent(ele,params=params_preprocess).strip() for ele in tqdm_notebook(annotated_df['message_text'],total=len(annotated_df))]

In [11]:
import random 
random.sample(list_sents,5)

['मदरसे   गुरुकुल    मदरसे  गुरुकुल  कारण',
 'प्रशासक समिति    जिस भारतीय जनता पार्टी के नेताओं ने अपना पूरा राजनीतिक जीवन दाँव पर लगाकर भारत की पवित्र भूमि में से एक अयोध्या  नगरी   जहां साक्षात तीन लोकों के स्वामी भगवान   श्री राम   का जन्म हुआ  उस पवित्र भूमि पर वर्षो से लगे एक आततायी मुस्लिम शासक बाबर द्वारा राममंदिर तोड़कर बनायी बाबरी मस्जिद रूपी कलंक को धोया   आज उसी भारतीय जनता पार्टी पर अगर कोई हिंदू होकर ये आरोप लगाए कि कि भारतीय जनता पार्टी के कारण राममंदिर निर्माण में विलंभ हो रहा है तो विश्वास मानिए एसे आरोप लगाने वाले व्यक्ति की मंदबुद्धी पर मुझे तरस आता है   अगर भारतीय जनता पार्टी अयोध्या में राममंदिर नहीं बनवाएगी तो क्या समाजवादी पार्टी बनवाएगी  जिसका मुखिया  मुलायम सिंह यादव  भरे मंच पर सीना ठोक कहते हैं कि हां मैंने मुसलमान भाईयों की खातिर कारसेवकों पर गोलियां चलवाई थीं  तो क्या आप उनके ही पुत्र अखिलेश यादव से राममंदिर निर्माण की आशा लगाए हैं जिसने अपने शासन काल में मुसलमानों के लिए गाजियाबाद में करोड़ो की लागत से एशिया के सबसे बड़े हज हाऊस का निर्माण करवाया   तो क्या 

In [20]:
count=[]
for sent in list_sents:
    count.append(len(tokenizer.tokenize(sent)))

#X_0 = np.array(list_sents,dtype='object')
# y_0 = np.array(annotated_df['one_fear_speech'])
# tokenizer = AutoTokenizer.from_pretrained(params['model_path'])
    
# X_train, X_train_length = encode_documents(X_0,params,tokenizer) 

In [29]:
count_greater=0
for ele in count:
    if(ele > (128*5)):
        count_greater+=1
count_greater

2030

In [25]:
count

[236,
 85,
 392,
 42,
 1154,
 1245,
 2141,
 153,
 340,
 985,
 246,
 30,
 209,
 570,
 542,
 1067,
 837,
 594,
 869,
 1014,
 895,
 49,
 667,
 1035,
 3217,
 45,
 86,
 666,
 1395,
 311,
 33,
 137,
 627,
 93,
 52,
 210,
 5030,
 539,
 1452,
 3111,
 588,
 1052,
 86,
 65,
 44,
 478,
 43,
 1478,
 491,
 1261,
 269,
 2313,
 308,
 1163,
 1262,
 402,
 1441,
 322,
 883,
 143,
 265,
 415,
 494,
 459,
 1012,
 608,
 1423,
 1275,
 201,
 534,
 890,
 784,
 438,
 389,
 887,
 634,
 1419,
 123,
 311,
 135,
 1622,
 432,
 776,
 1189,
 805,
 876,
 819,
 250,
 480,
 428,
 946,
 832,
 884,
 85,
 197,
 44,
 138,
 1726,
 646,
 1715,
 323,
 3072,
 3307,
 202,
 60,
 645,
 1178,
 229,
 858,
 584,
 2242,
 2228,
 53,
 640,
 1551,
 264,
 596,
 221,
 2835,
 975,
 940,
 200,
 351,
 788,
 47,
 2251,
 74,
 450,
 1633,
 396,
 89,
 789,
 53,
 178,
 667,
 962,
 42,
 30,
 45,
 1084,
 945,
 81,
 1472,
 499,
 116,
 977,
 497,
 1244,
 640,
 1760,
 3063,
 854,
 907,
 2612,
 551,
 50,
 231,
 761,
 1118,
 53,
 37,
 461,
 4325,
 239,
 

In [ ]:
train_dataloader=return_dataloader(X_train,y_0,params,is_train=True)


In [ ]:
train_dataloader

In [ ]:
 for step, batch in tqdm(enumerate(train_dataloader)):

        b_input_ids = batch[0]
        b_labels = batch[1]
        break

In [ ]:
b_input_ids.shape

In [ ]:
b_input_ids[0][:20,1]

In [31]:
model=select_transformer_model(params['transformer_type'],params['model_path'],params)

I0924 13:27:07.329630 140120072415040 connectionpool.py:700] Starting new HTTPS connection (1): s3.amazonaws.com
I0924 13:27:08.513124 140120072415040 connectionpool.py:700] Starting new HTTPS connection (1): cdn.huggingface.co


{'model_path': 'bert-base-multilingual-cased', 'max_length': 128, 'batch_size': 6, 'weights': [1.0, 1.0], 'data_path': '../Data/New_Data_15-06-2020/Fearspeech_data_final.pkl', 'max_sentences_per_doc': 10, 'transformer_type': 'lstm_transformer', 'device': 'cuda', 'learning_rate': 2e-05, 'epsilon': 1e-08, 'random_seed': 2, 'epochs': 5}


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing DocumentBERTLSTM: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing DocumentBERTLSTM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DocumentBERTLSTM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DocumentBERTLSTM were not initialized from the model checkpoint at bert-base-multilingual-cased and are new

In [36]:
model.classifier.parameters

<bound method Module.parameters of Sequential(
  (0): Dropout(p=0.1, inplace=False)
  (1): Linear(in_features=768, out_features=2, bias=True)
  (2): Tanh()
)>